In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from keras.models import load_model
import time

model = load_model('model.h5')

cap = cv2.VideoCapture(2)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    print(frame.shape)

    # Görüntüyü ikiye bölmek için boyutları alın
    height, width, _ = frame.shape
    half_width = width // 2

    # Görüntüyü ikiye bölmek
    left_half = frame[:, :half_width]
    right_half = frame[:, half_width:]

    img1 = cv2.resize(left_half, (512, 512))
    img2 = cv2.resize(right_half, (512, 512))
    img1 = np.expand_dims(img1, axis=0)
    img2 = np.expand_dims(img2, axis=0)

    predictions = model.predict(img1)
    predictions2 = model.predict(img2)

    x1, y1, x2, y2 = predictions[0].astype(float)
    x1 = x1*512
    y1 = y1*512
    x2 = x2*512
    y2 = y2*512

    x1 = int(x1)
    x2 = int(x2)
    y1 = int(y1)
    y2 = int(y2)

    x11, y11, x22, y22 = predictions2[0].astype(float)
    x11 = x11*512
    y11 = y11*512
    x22 = x22*512
    y22 = y22*512

    x11 = int(x11)
    x22 = int(x22)
    y11 = int(y11)
    y22 = int(y22)
    
    cv2.rectangle(left_half, (x1, y1), (x2, y2), (255, 0, 0), 2)
    cv2.putText(left_half, str(1), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
    cv2.rectangle(right_half, (x11, y11), (x22, y22), (255, 0, 0), 2)
    cv2.putText(right_half, str(2), (x11, y11-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)

    combined_frame_horizontal = cv2.hconcat([left_half, right_half])


    # Görüntü boyutlarını alın
    height, width, _ = frame.shape

    # Tam ortadan dikey çizgi çizme
    thickness = 1
    cv2.line(combined_frame_horizontal, (width // 2, 0), (width // 2, height), (255, 255, 255), thickness)
    cv2.line(combined_frame_horizontal, (0, height // 2), (width, height // 2), (255, 255, 255), thickness)

    cv2.imshow('Kamera',combined_frame_horizontal)
    time.sleep(0.1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
